In [5]:
import pandas as pd

In [29]:
df = pd.read_excel(r".\FSIS-Recall-Summary-2014.xlsx")


In [30]:
df.sample(10)

,CY 2014 Recalls,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
39,2014-06-25 00:00:00,039-2014,II,Sausage Products,Undeclared Allergen,1761
17,2014-02-26 00:00:00,017-2014,I,Liver Pâté,Other,1282
26,2014-04-25 00:00:00,026-2014,II,Barbecue Pork Products,Undeclared Allergen,350
90,2014-12-27 00:00:00,090-2014,I,Pork products,Undeclared Allergen,83666
29,2014-05-16 00:00:00,029-2014,III,Pork Products,Other,449000
92,2014-12-29 00:00:00,092-2014,I,Beef and Pork Products,Undeclared Allergen,38400
85,2014-12-19 00:00:00,085-2014,III,Spicy Sausage Pasta Products,Other,2719
56,2014-09-06 00:00:00,056-2014,II,"Fresh, Raw Chicken Products",Processing Defect,720
20,2014-04-02 00:00:00,020-2014,I,Fettuccini Alfredo with Chicken Products,Undeclared Allergen,633
68,2014-10-07 00:00:00,068-2014,I,Meat and Poultry Products,Listeria monocytogenes,62488
